# Wide Residual Networks CIFAR 10 Example

## Import and process the data

In [ ]:
from keras import utils
from keras.datasets import cifar10

classes = 10
img_rows, img_cols, img_channels = 32, 32, 3

(trainX, trainY), (testX, testY) = cifar10.load_data()

# Rescale and change data type of images
trainX = trainX.astype('float32') / 255.
testX = testX.astype('float32') / 255.

# Convert labels to one-hot
trainY = utils.to_categorical(trainY, classes)

print("Train shape:", trainX.shape)
print("Test shape:", testX.shape)

## Data generators

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_generator = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    width_shift_range=4./img_cols,
    height_shift_range=4./img_rows,
    fill_mode='reflect',
    horizontal_flip=True)
train_generator.fit(trainX, seed=0)

test_generator = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True)
test_generator.fit(trainX, seed=0)

## Instantiate the Wide Residual Network

In [ ]:
from deep_models import wide_residual_network as wrn
from keras.utils import plot_model

n = 4  # 6 * n + 4 is the depth
k = 10  # k is the width
dropout = 0.3

fname = 'WRN-{}-{}{}'.format(N * 6 + 4, k, '-dropout' if dropout > 0 else '')
model_path = '{}.h5'.format(fname)

# Create the model
model = wrn.build_model((img_cols, img_rows, img_channels), classes=classes, n=n, k=k, dropout=dropout)

## Train the model

In [ ]:
import keras.callbacks as callbacks
from keras.optimizers import SGD

epochs = 200
batch_size = 64


def scheduler(epoch, lr):
    if epoch in [60, 120, 160]:
        lr *= 0.2
    return lr


sgd = SGD(lr=0.1, momentum=0.9, decay=0.0, nesterov=True)
model.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=["acc"])

# Train the model
history = model.fit_generator(
    train_generator.flow(trainX, trainY, batch_size=batch_size), 
    steps_per_epoch=len(trainX) / batch_size, 
    epochs=epochs,
    validation_data=test_generator.flow(testX, testY),
    callbacks=[
        callbacks.ModelCheckpoint(model_path, monitor="val_acc", save_best_only=True),
        callbacks.LearningRateScheduler(scheduler)
    ])

## Save the final version

In [ ]:
import time

model.save('{}-{}.h5'.format(fname, time.time()))

## Print model score

In [ ]:
from keras.models import load_model
model = load_model(model_path)

metrics = model.evaluate_generator(test_generator.flow(valX, valY, shuffle=False))

accuracy = metrics[1] * 100
error = 100 - accuracy
print("Accuracy : ", accuracy)
print("Error : ", error)

## Show the training history

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# summarize history for accuracy  
plt.figure(1)  
plt.plot(history.history['acc'])  
plt.plot(history.history['val_acc'])  
plt.title('model accuracy')  
plt.ylabel('accuracy')  
plt.xlabel('epoch')  
plt.legend(['train', 'test'], loc='upper left')  
   
# summarize history for loss  
plt.figure(2)
plt.plot(history.history['loss'])  
plt.plot(history.history['val_loss'])  
plt.title('model loss')  
plt.ylabel('loss')  
plt.xlabel('epoch')  
plt.legend(['train', 'test'], loc='upper left')  
plt.show()